In [1]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
from sklearn.model_selection import train_test_split

reference: https://github.com/amaiya/ktrain

# Import a Small Dataset
There are 200 items in the dataset. Lable 0 means Fakenews, and Lable 1 means Truenews.

In [2]:
df_200 = pd.read_csv('dataset-200.csv')

In [3]:
df = df_200.reindex(np.random.permutation(df_200.index))
df.head()

,title,text,subject
113,Factbox: Republican U.S. senators to watch in ...,(Reuters) - Republican Senator Rand Paul said ...,1
148,House ethics panel investigating allegations a...,WASHINGTON (Reuters) - The House of Representa...,1
8,"AG Sessions Persuaded Trump To End DACA, Then...",It s certainly no secret that Jefferson Beaure...,0
94,Steve Mnuchin‚Äôs Spoiled Wife Just Went Apes...,"Donald Trump s Treasury Secretary, former hedg...",0
11,Obama Pens STUNNING Response To Trump‚Äôs Col...,Trump is working his tail off to undo absolute...,0


In [4]:
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
    df['title'] = df['title'].str.replace(char, ' ')

In [5]:
df['title']=df['title'].str.lower()
df.head()

,title,text,subject
113,factbox republican u s senators to watch in ...,(Reuters) - Republican Senator Rand Paul said ...,1
148,house ethics panel investigating allegations a...,WASHINGTON (Reuters) - The House of Representa...,1
8,ag sessions persuaded trump to end daca then...,It s certainly no secret that Jefferson Beaure...,0
94,steve mnuchin‚äôs spoiled wife just went apes...,"Donald Trump s Treasury Secretary, former hedg...",0
11,obama pens stunning response to trump‚äôs col...,Trump is working his tail off to undo absolute...,0


In [6]:
categories = [1, 0]

y = df['subject'].values
X = df['title']

X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y,test_size=0.30, random_state=0)

In [7]:
X_train = X_train.to_numpy()
# y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
# y_test = y_test.to_numpy()

print("X_train.shape: ", X_train.shape)
print("X_train type: ", type(X_train))
print("X_test.shape: ", X_test.shape)
print("y_train.shape: ", y_train.shape)
print("y_test.shape: ", y_test.shape)

X_train.shape:  (140,)
X_train.shape:  <class 'numpy.ndarray'>
X_test.shape:  (60,)
y_train.shape:  (140,)
y_test.shape:  (60,)


In [10]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=categories,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [11]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)

Is Multi-Label? False
maxlen is 350
done.


In [12]:
learner.fit_onecycle(2e-5, 10)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
24/24 [==============================] - 159s 6s/step - loss: 0.6724 - accuracy: 0.6151
Epoch 2/10
24/24 [==============================] - 141s 6s/step - loss: 0.5313 - accuracy: 0.8450
Epoch 3/10
24/24 [==============================] - 142s 6s/step - loss: 0.2540 - accuracy: 0.9341
Epoch 4/10
24/24 [==============================] - 143s 6s/step - loss: 0.0966 - accuracy: 0.9831
Epoch 5/10
24/24 [==============================] - 139s 6s/step - loss: 0.0288 - accuracy: 1.0000
Epoch 6/10
24/24 [==============================] - 139s 6s/step - loss: 0.0105 - accuracy: 1.0000
Epoch 7/10
24/24 [==============================] - 138s 6s/step - loss: 0.0053 - accuracy: 1.0000
Epoch 8/10
24/24 [==============================] - 139s 6s/step - loss: 0.0052 - accuracy: 1.0000
Epoch 9/10
24/24 [==============================] - 189s 8s/step - loss: 0.0034 - accuracy: 1.0000
Epoch 10/10
24/24 [===========================

We can use the learner.validate method to test our model against the Test set.

In [16]:
learner.validate(val_data=(x_test, y_test), class_names=categories)

              precision    recall  f1-score   support

           1       0.93      0.87      0.90        30
           0       0.88      0.93      0.90        30

    accuracy                           0.90        60
   macro avg       0.90      0.90      0.90        60
weighted avg       0.90      0.90      0.90        60



array([[26,  4],
       [ 2, 28]])